In [7]:
!pip install easyocr

In [9]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...


True

In [10]:
import easyocr
reader = easyocr.Reader(['en'])

from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [11]:
def extract_text_from_image(image_path):
    """
    Extracts visible text from an image using EasyOCR.
    """
    results = reader.readtext(image_path, detail=0)
    extracted_text = " ".join(results)
    return extracted_text


In [12]:
SCAM_KEYWORDS = [
    # Urgency / Threat
    "verify", "account blocked", "urgent", "limited time", "immediate action", "final warning",
    "update now", "act fast", "deadline", "security alert", "deactivation", "unauthorized access",

    # Refund / Payment Scams
    "refund", "payment failed", "transaction failed", "pay immediately", "receive money",
    "get back", "pending amount", "recharge issue", "loan approved", "claim refund", "wrong payment",

    # Clickbait / Phishing Triggers
    "click here", "open link", "tap now", "scan to receive", "visit now", "login here", "redeem now",
    "login quickly", "scan QR", "follow instructions",

    # Free / Prize / Reward
    "win", "reward", "free", "gift", "bonus", "offer ends", "cashback", "you are selected", 
    "lucky draw", "free recharge", "prize claim",

    # OTP / Fake Verification
    "OTP", "activation", "your account", "bank issue", "validate", "confirm your identity",
    "update KYC", "kyc expired", "complete KYC", "aadhaar verify", "PAN verification", "reset password",

    # UPI/BharatPay Specific
    "UPI refund", "UPI support", "BharatPay", "Scan this code", "QR payment", "check transaction",
    "pay ₹1 to verify", "request sent", "account credited",

    # Psychological pressure
    "emergency", "family issue", "accident", "hospital", "doctor fee", "help me", "need money urgently",
    "money stuck", "I’m stranded", "mom in hospital", "ambulance"

]


In [15]:
def context_score(text,SCAM_KEYWORDS):
    if not text:
        return {
            'text': '',
            'scam_keywords_count': 0,
            'sentiment': {},
            'context_score': 0.0,
            'suspicious': False
        }

    # Normalize
    text_lower = text.lower()

    # Keyword scam score
    scam_hits = [kw for kw in SCAM_KEYWORDS if kw in text_lower]
    scam_score = len(scam_hits)

    # Sentiment
    sentiment = sia.polarity_scores(text)

    # Combine scores
    context_score = (scam_score * 0.5) + (sentiment['neg'] * 5)  # Weight keywords + negative tone
    suspicious = context_score > 2  

    return {
        'text': text,
        'scam_keywords': scam_hits,
        'scam_keywords_count': scam_score,
        'sentiment': sentiment,
        'context_score': round(context_score, 2),
        'suspicious': suspicious
    }


In [14]:
def explain_prediction(upi_id, context_result, anomaly_score):
    if context_result['suspicious'] and anomaly_score == -1:
        reason = f"The system flagged this as suspicious because the message contains keywords like {context_result['scam_keywords']} and had a high negative sentiment."
    elif context_result['suspicious']:
        reason = f"The context around this message raised red flags due to keywords like {context_result['scam_keywords']} and emotional tone."
    elif anomaly_score == -1:
        reason = f"The UPI ID '{upi_id}' shows unusual behavior compared to common patterns, based on our anomaly detection model."
    else:
        reason = f"The UPI ID '{upi_id}' appears normal based on both its behavior and message context."
    
    return reason


In [ ]:
# import streamlit as st

# st.header("Model Explanation")

# # After your model makes a prediction
# last_result = {
#     'upi_id': 'abc@upi',
#     'context': context_score("Verify your account immediately or get blocked."),
#     'model_pred': -1
# }

# explanation = explain_prediction(last_result['upi_id'], last_result['context'], last_result['model_pred'])
# st.info(explanation)

# # Mini chat interface
# st.subheader("Ask UPIShield Bot 👇")
# user_question = st.text_input("Ask me why this was flagged...")

# if user_question:
#     bot_reply = mini_bot(user_question, last_result)
#     st.success(bot_reply)

# # Optional Feedback
# feedback = st.radio("Was this explanation helpful?", ["Yes", "No"])
# if st.button("Submit Feedback"):
#     st.success("Thanks! Your feedback helps improve the model.")
